In [29]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2

# Path ke folder dataset
dataset_folder = 'images/'

# Path ke file styles.csv
styles_file = 'styles.csv'

# Membaca data dari file styles.csv
data = pd.read_csv(styles_file)

# Membaca gambar dan label dari folder dataset
images = []


# Memuat label ke dalam variabel
label_csv = data['articleType']


# Mengonversi label FashionImage menjadi tipe data string
label_csv = label_csv.astype(str)

# Menggabungkan gambar dari folder 'images' dengan dataset MNIST
image_folder = 'images/'

for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Mengubah gambar menjadi skala abu-abu (grayscale)
    image = cv2.resize(image, (28, 28))  # Menyesuaikan dimensi gambar menjadi 28x28
    image = np.expand_dims(image, axis=-1)  # Menambahkan dimensi kanal (1) pada gambar
    images.append(image)

# Konversi data gambar menjadi array numpy
images_data = np.array(images)

In [30]:
# Mengubah label menjadi angka menggunakan LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(label_csv)
num_classes = len(label_encoder.classes_)

#mengetahui jumlah data
print('jumlah gambar ', images_data.shape)
print('jumlah label ', label_csv.shape)


jumlah gambar  (44441, 28, 28, 1)
jumlah label  (44441,)


In [36]:
# Membagi dataset menjadi data training, data test, dan data validasi
x_train, x_test, y_train, y_test = train_test_split(images_data, labels, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

# Normalisasi pixel gambar menjadi rentang [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_val = x_val.astype('float32') / 255

# Membangun arsitektur model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
]) 

# Kompilasi model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Pelatihan model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_val, y_val))

# Evaluasi model pada data test
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)


Epoch 1/10
556/556 [==============================] - 133s 237ms/step - loss: 2.5716 - accuracy: 0.3571 - val_loss: 1.5582 - val_accuracy: 0.5735
Epoch 2/10
556/556 [==============================] - 133s 239ms/step - loss: 1.5606 - accuracy: 0.5766 - val_loss: 1.2554 - val_accuracy: 0.6578
Epoch 3/10
556/556 [==============================] - 133s 238ms/step - loss: 1.2915 - accuracy: 0.6407 - val_loss: 1.0563 - val_accuracy: 0.6963
Epoch 4/10
556/556 [==============================] - 138s 249ms/step - loss: 1.1409 - accuracy: 0.6778 - val_loss: 0.9629 - val_accuracy: 0.7208
Epoch 5/10
556/556 [==============================] - 133s 238ms/step - loss: 1.0437 - accuracy: 0.6982 - val_loss: 0.8977 - val_accuracy: 0.7323
Epoch 6/10
556/556 [==============================] - 130s 233ms/step - loss: 0.9682 - accuracy: 0.7150 - val_loss: 0.8686 - val_accuracy: 0.7334
Epoch 7/10
556/556 [==============================] - 131s 235ms/step - loss: 0.9110 - accuracy: 0.7296 - val_loss: 0.8059 -

In [ ]:
# Simpan model
model.save('image_classification_model.h5')
